In [30]:
import pandas as pd
import os
import multiprocessing
import threading

In [31]:
pathToCSV='/home/livelab/Desktop/CELA-VideoClassification/VideoDataProcessing/framecsv'
CSVName='{}frame.csv'
subjects=['S1','S2','S3','S4','S5','S6','S7']
CSVHeader=['PROCEDURE','Original_Start_Time','Original_Stop_Time']
adjustedStartingFrame={'S1':24030/30*29.97,'S2':30060/30*29.97,'S3':79650/30*29.97,
                         'S4':39600/30*29.97,'S5':24120/30*29.97,'S6':28320/30*29.97,'S7':34800/30*29.97}
TTVList=['Training', 'Test', 'Validation']
cpCmd='cp {} {}'
pathToDataList=['/media/livelab/Samsung_T5/MedicalProdecureFrames/Vid2FrameS1',
               '/media/livelab/Storage/Vid2FrameS2',
               '/media/livelab/Samsung_T5/MedicalProdecureFrames/Vid2FrameS3',
               '/media/livelab/Storage/Vid2FrameS4',
               '/media/livelab/Samsung_T5/MedicalProdecureFrames/Vid2FrameS5',
               '/media/livelab/Storage/Vid2FrameS6',
               '/media/livelab/Samsung_T5/MedicalProdecureFrames/Vid2FrameS7',]
pathToDestination='/media/livelab/Storage/NewFrameData'

In [32]:
def makedir(dirName):
    if not os.path.exists(dirName):
        cmd='mkdir "{}"'
        os.system(cmd.format( dirName))

In [33]:
df=pd.read_csv(os.path.join(pathToCSV,CSVName.format(subjects[6])))
print('length',len(df[CSVHeader[0]]))
df[CSVHeader[0]]

length 197


0                        CLAP
1                 ORAL AIRWAY
2                     BAGGING
3                  INTUBATION
4                  INTUBATION
5                 ORAL AIRWAY
6                 KING AIRWAY
7                 ORAL AIRWAY
8      SWAB AREA WITH ALCOHOL
9             DRAW MEDICATION
10          IM ADMINISTRATION
11            CHEST-TUBE PREP
12                 CHEST-TUBE
13                   SUTURING
14     SWAB AREA WITH ALCOHOL
15            DRAW MEDICATION
16     SWAB AREA WITH ALCOHOL
17          IM ADMINISTRATION
18                  ECG LEADS
19                   PULSE-OX
20        BLOOD-PRESSURE CUFF
21             VITAL CHECKING
22          COMBAT TOURNIQUET
23                  SPLINTING
24               COMBAT GAUZE
25            WRAP HEAD WOUND
26          CPR (COMPRESSION)
27               CPR (BREATH)
28          CPR (COMPRESSION)
29               CPR (BREATH)
                ...          
167    SWAB AREA WITH ALCOHOL
168    SWAB AREA WITH ALCOHOL
169       

In [34]:
def padNameWithQuotes(name):
    return '"'+name+'"'

In [35]:
def categorizedFrameData(ID):
    print(ID)
    if ID< 5:
        TTVid=0
    elif ID == 5:
        TTVid=1
    else:
        TTVid=2
    pathToTTV=os.path.join(pathToDestination,TTVList[TTVid])
    makedir(pathToTTV)
    #prepare CSV
    df=pd.read_csv(os.path.join(pathToCSV,CSVName.format(subjects[ID])))
    
    #read in list data from Data folder
    pathToData=pathToDataList[ID]
    listOfData=os.listdir(pathToData)
    listOfData.sort()
    counterInCSV=0
    for data in listOfData:
        if counterInCSV == len(df[CSVHeader[0]]):
            continue
        frameNo=int(data[4:10])
        if(frameNo%40000==0):
            print (frameNo, counterInCSV)
        categoryStart=df[CSVHeader[1]][counterInCSV]
        categoryEnd=df[CSVHeader[2]][counterInCSV]
        categoryName=df[CSVHeader[0]][counterInCSV]
        adjustedStart=int(float(categoryStart)+adjustedStartingFrame[subjects[ID]])
        adjustedEnd=int(float(categoryEnd)+adjustedStartingFrame[subjects[ID]])
        #if smaller than starting time, then ignore
        if(frameNo<adjustedStart):
            continue
        #if in between the time, then copy the frame image to destination given by 'PROCEDURE'
        elif(frameNo>=adjustedStart and frameNo<=adjustedEnd):
            fullPathToData=os.path.join(pathToData,data)
            pathToDesCate=os.path.join(pathToTTV, categoryName)
            makedir(pathToDesCate)
            os.system(cpCmd.format(padNameWithQuotes(fullPathToData),padNameWithQuotes(pathToDesCate)))
            if frameNo==adjustedEnd:
                counterInCSV+=1

In [36]:
pool = multiprocessing.Pool(2)
N = 7
_=pool.map(categorizedFrameData,[6])
pool.close()
pool.join()

6
40000 2
80000 13
120000 55
160000 84
200000 110
240000 141
280000 172
320000 195


In [3]:
a='qwer try'
b='ret wwe'

os.path.join(a,b)

'qwer try/ret wwe'

In [10]:
20%8

4